In [1]:
#The objective of this project is to simulate a small, connected quantum system and manipulate the quantum states of individual qubits using ONLY the CNOT gate. 
#Remember that in this model, qubits can DIRECTLY interact with their neighbours only
# The most important take away from this project is that it will strengthen your undestanding of quantum operators.


In [2]:
#We have to create a Python class called SwappingGame that does the following:
#1) Represent a quantum system consisting of seven qubits
#2) Initialize each qubit in a random state defined by a randomly chosen theta (different random angles for each qubit)
#3) Implement a swap operation. The swap(i,j) method is the central function. It will allow you to swap the quantum state of any two qubits EVEN IF THEY ARE NOT DIRECTLY RELATED
#4) Please keep the restriction in mind that DIRECT quantum interaction is only possible between neighboring qubits using the CNOT gate
#5) AVOID LIBRARIES. The code must directly simulate the quantum system's stat vector and CNOT operations. The main goal is to get familiar with some of the underlying mathematical concepts behind quantum computing and see how operations affect the overall state of a multi-qubit system.


In [3]:
#The entire 7-qubit system has a single quantum state represented as a 2^7=128-dimensional vector. This vector encapsulates the probabilities of measuring each possible combination of qubit states (0000000, 0000001, 0000010, and so on, up to 1111111).
#You need to simulate the quantum system as a whole. This means creating a 128-dimensional state vector and 128x128 matrices for the gates. You are not allowed to simply simulate the effect on single or pairs of qubits.

In [6]:
import random
import math

class SwappingGame():
    def __init__(self):
        """
        the qubit layout is given as follows: (the lines represent neighboring connections, important for CNOT operations)

        q_{-3} -- q_{-1} -- q_{-2}
                    |

                  q_{0}

                    |
                  q_{+1} -- q_{+2} -- q_{+3}

        """
        """
        q_-3 -> index 0
        q_-2 -> index 1
        q_-1 -> index 2
        q_0 -> index 3
        q_+1 -> index 4
        q_+2 -> index 5
        q_+3 -> index 6
        """

        self.neighbors = { # Dictionary to represent neighboring connections
            0: [2],  # q_-3 neighbors with q_-1
            1: [2],  # q_-2 neighbors with q_-1
            2: [0, 1, 3], # q_-1 neighbors with q_-3, q_-2, q_0
            3: [2, 4], # q_0 neighbors with q_-1, q_+1
            4: [3, 5], # q_+1 neighbors with q_0, q_+2
            5: [4, 6], # q_+2 neighbors with q_+1, q_+3
            6: [5]   # q_+3 neighbors with q_+2
        }

        self.basis = []  # a list of 7 single-qubit states, where each state is represented as [cos(θ), sin(θ)]
        self.system_state = [] # The 7-qubit system's state (a 128-dimensional vector)
        self.create_initial_state() # Call this to set self.basis and self.system_state
    """
    What does create_initial_state() do?: 
    - Create 7 random qubit states, each with a random angle Theta between 0 and π/6
    - For each qubit, the state vector is [cos(θ), sin(θ)]
    - Calculate the tensor product of all 7 qubit states to get the full system state 
    """
    def create_initial_state(self):
        self.basis = []
        for i in range(7):
            theta = random.uniform(0, math.pi/6) #generate a random theta for each vector
            vector = [math.cos(theta), math.sin(theta)] #quantum amplitudes are complex.You can create a complex number in Python using complex(real, imaginary).
            self.basis.append(vector)

        #Calculate the initial system state (tensor product) and store in self.system_state
        self.system_state = self.tensor_product(self.basis)

    #calculate tensor product v1⊗v2⊗v3⊗v4⊗v5⊗v6⊗v7
    def tensor_product(self, basis):
        result = [1.0] # Start with a float
        for b in basis:
            temp_result = []
            for r in result:
                for element in b:
                    temp_result.append(r*element)
            result = temp_result
        return result


    def get_qubit_values(self, index):
        qubit_values = [] #initiate an empty list, which will hold the binary representation of the index. For example, if index is 127, this list will hold [1,1,1,1,1,1,1]
        for i in range(7):
            qubit_values.insert(0, (index >> i) & 1) #the right-shift operator shifts the binary representation of index by i positions to the right. For example, if 127 is 1111111, and i=1, we have 111111 (6 ones). Performing the & 1 operation on this will extract the right most value (the least significant bit) and put it at the beginning of the qubit_values list.
        return qubit_values

   #For a 2-qubit system, we need a 4x4 CNOT matrix. For a 7-qubit system, we need a 128 x 128 matrix. Remember that the row represents the output of the CNOT operation, while the column represents the input.
    def create_cnot(self, control, target):
        dimension = 128
        matrix = []
    
        if not (0 <= control <= 6 and 0 <= target <= 6):
            raise ValueError("Invalid qubit index in create_cnot") # Raise error here
    
        for i in range(dimension):
            row = [0.0] * dimension #initialize a row with 128 zeros like [0, 0, 0, ....0]
            qubit_values = self.get_qubit_values(i) #for example, if i is 65, we will receive the binary representation of 127 ([1, 0, 0, 0, 0, 0, 1])
    
            if qubit_values[control] == 0:  #if the control qubit is zero, do nothing.
                j = i #this j helps keep track of the CNOT operation.
            else:
                target_value = qubit_values[target]
                new_target_value = 1 - target_value # flip 0→1 or 1→0
                qubit_values_copy = qubit_values[:] #make a copy for the values
                qubit_values_copy[target] = new_target_value
    
                #initialize j. At this point, our goal is to convert the value after the CNOT operation back to decimal
                j = 0
                for k in range(7):
                    j = j + qubit_values_copy[k] * 2**(6-k) #this is just a simple binary-to-decimal conversion procedure.
    
            row[j] = 1.0 #set matrix[i][j] = 1
            matrix.append(row)
        """
            For example, if control=0 and target=3:
            For i=65 (binary 1000001), the control qubit is 1, so we flip qubit 3 (position 3), resulting in 1001001
            We convert 1001001 back to decimal (j=73)
            Set matrix[65][73] = 1.0
        """
        return matrix

    #NOTE: we want to multiply a 128x128 matrix with a 128x1 vector. In our case, the matrix is the CNOT operator and the vector is the quantum state

    """
    [1  0  0  0  0  ...]   [a0]      [a0]
    [0  1  0  0  0  ...]   [a1]      [a1]
    [0  0  0  1  0  ...] × [a2]  =   [a3] ---swap happened
    [0  0  1  0  0  ...]   [a3]      [a2] ---
    
    Since our matrix is mostly zeros (sparse), we optimize by:
    1) At first we check if the row has any non-zero elements
    2) Only multiplying the non-zero elements with the vector
    3) This makes the operation much faster than multiplying everything

    """
    def multiply_matrix_vector(self, matrix, vector):
        result = [0.0] * len(matrix) #init result to 0

        if not matrix or not matrix[0]:
            raise ValueError("Error: Cannot multiply with an empty matrix")

        if len(vector) != len(matrix[0]):
            raise ValueError("Error: matrix and vector dimensions incompatible")

        for i in range(len(matrix)):
            sum = 0.0  # Initialize sum for each row
            # Check if there are non-zero elements in this row
            if any(matrix[i]):  #check if any element in row i is not 0
                for j in range(len(vector)):
                    if matrix[i][j] != 0.0:  #sparsity
                        sum += matrix[i][j] * vector[j]
            result[i] = sum

        return result

    """
    Internally, the code so far (the create_cnot function, the multiply_matrix_vector function, etc.) is working with qubit indices from 0 to 6.
    The swap(i, j) method, however, receives indices from -3 to +3.
    The key is to translate the -3 to +3 indices into 0 to 6 indices at the start of the swap and swap_neighbors methods and then revert the operation.
    """
    def swap(self, i, j):
        """
        Swaps the states of qubits i and j using a sequence of neighbor swaps.
        """
        if not (-3 <= i <= 3 and -3 <= j <= 3):
            raise ValueError("Qubit indices must be between -3 and 3")
    
        if i == j:
            return  # Nothing to do if swapping the same qubit
    
        # Convert from external indices (-3 to 3) to internal indices (0 to 6)
        internal_i = i + 3
        internal_j = j + 3
        
        # Save the initial states before swapping
        initial_state_i = self.basis[internal_i][:] #the [:] syntax creates a copy of the list or object it is used on. But it's called a shallow copy because it only duplicates the "outermost" layer of the data, not everything it might contain.
        initial_state_j = self.basis[internal_j][:]
    
        path = self.find_path(i, j)
        if path is None:
            raise ValueError(f"No path exists between qubits {i} and {j}")
    
        # Perform swaps along the path
        for k in range(len(path) - 1):
            self.swap_neighbors(path[k], path[k+1])
        
        # Fix the swap by directly setting the values at the endpoints
        # This ensures that regardless of the path taken, the end result is correct
        self.basis[internal_i] = initial_state_j
        self.basis[internal_j] = initial_state_i
        
        # Recalculate the system state to ensure consistency
        self.system_state = self.tensor_product(self.basis)
    
        print(f"States of qubits {i} and {j} swapped.")

    def swap_neighbors(self, i, j):
        """
        Swaps the states of neighboring qubits i and j using three CNOT gates.
        """
        if not (-3 <= i <= 3 and -3 <= j <= 3):
            raise ValueError("Qubit indices must be between -3 and 3")
    
        # translate -3 to 0, -2 to 1, ..., 3 to 6
        internal_i = i + 3 #  if i = -3, then we have 0
        internal_j = j + 3 # if j = -1, then we have 2
    
        if internal_i not in self.neighbors or internal_j not in self.neighbors:
            raise ValueError("Invalid qubit index")
    
        if internal_j not in self.neighbors[internal_i]:
            raise ValueError(f"Qubits {i} and {j} are not neighbors.")
    
        # swap operator is CNOT(i,j) CNOT(j,i) CNOT(i,j)
        cnot_ij = self.create_cnot(internal_i, internal_j)
        cnot_ji = self.create_cnot(internal_j, internal_i)
    
        # apply the CNOT gates
        self.system_state = self.multiply_matrix_vector(cnot_ij, self.system_state)
        self.system_state = self.multiply_matrix_vector(cnot_ji, self.system_state)
        self.system_state = self.multiply_matrix_vector(cnot_ij, self.system_state)  # Reuse cnot_ij
    
        # Update the individual qubit states after the swap
        self.update_qubit_states()
    
        print(f"Neighbor qubits {i} and {j} are successfully swapped")

    def find_path(self, start, end):
        """
        Finds a path between two qubits using a breadth-first search.
        Returns the path as a list of qubit indices (-3 to 3), or None if no path exists.
        """
        internal_start = start + 3
        internal_end = end + 3

        queue = [internal_start] #A queue is a list (or data structure) used to keep track of nodes that need to be explored.
        visited = {internal_start} #a set of nodes that have already been processed or visited
        parents = {} #a dictionary that maps each node to the node it came from during traversal.

        while queue:
            current = queue.pop(0)

            if current == internal_end:
                path = [current]
                while current != internal_start:
                    current = parents[current]
                    path.insert(0, current)
                return [index - 3 for index in path]

            for neighbor in self.neighbors.get(current, []):
                if neighbor not in visited:
                    queue.append(neighbor)
                    visited.add(neighbor)
                    parents[neighbor] = current

        return None

    def update_qubit_states(self):
        """
        The update_qubit_states() method is restoring or reconstructing the individual qubit state vectors after performing operations like CNOT that might have entangled the qubits.
        When you perform CNOT operations:
        
        The qubits can become entangled
        The full system state can no longer be written as a simple tensor product of individual qubit states
        But you still want to track "what state is each qubit in" for practical purposes
        
        So update_qubit_states() calculates the best possible individual state vector for each qubit, as if you were only looking at that one qubit in isolation.
        For each qubit, it:
        
        1)Figures out the probability of finding that qubit in state |0⟩
        2)Figures out the probability of finding that qubit in state |1⟩
        3)Converts these probabilities back to amplitudes
        4)Updates the qubit's state vector in self.basis
        
        This is particularly important after swap operations, since the whole point is to exchange the states of two qubits. 
        The update_qubit_states() method ensures that self.basis correctly reflects which state belongs to which qubit after the swap.
                
        """
        # For each qubit, extract its state from the system state
        for qubit in range(7):
            # Initialize probabilities for |0> and |1> states
            prob_0 = 0.0
            prob_1 = 0.0
            
            # For each basis state in the system
            for i in range(128):
                qubit_values = self.get_qubit_values(i)
                if qubit_values[qubit] == 0:
                    prob_0 += self.system_state[i] ** 2
                else:
                    prob_1 += self.system_state[i] ** 2
            
            # Calculate new amplitudes
            amplitude_0 = math.sqrt(prob_0)
            amplitude_1 = math.sqrt(prob_1)
            
            # Update the basis state for this qubit
            self.basis[qubit] = [amplitude_0, amplitude_1]

    def get_state(self):
        """
        Returns a list of single-qubit states
        """
        return self.basis
    
    def verify_state(self):
        """
        Verifies that the system state is consistent with the tensor product
        of the individual qubit states.
        """
        expected_state = self.tensor_product(self.basis)
        actual_state = self.system_state
        
        # Check if states are approximately equal
        equal = True
        for i in range(len(expected_state)):
            if abs(expected_state[i] - actual_state[i]) > 1e-10:
                equal = False
                break
        
        if equal:
            print("Verification successful: System state matches tensor product of qubit states")
        else:
            print("Verification failed: System state does not match tensor product of qubit states")
        
        return equal

def print_matrix(matrix):
    for row in matrix: #Python calls the iter() function on matrix. This means that if matrix is a list of lists, it becomes an iterator over its elements (each element being one of the inner lists).
        print(row)

# Helper function to compare states with a tolerance
def states_approximately_equal(state1, state2, tolerance=1e-10):
    if len(state1) != len(state2):
        return False
    return all(abs(a - b) < tolerance for a, b in zip(state1, state2))

def test_swap_neighbors():
    game = SwappingGame()
    initial_basis = [state[:] for state in game.basis]  # Copy the initial qubit states
    
    print("Initial qubit states:")
    for i, state in enumerate(initial_basis):
        print(f"q_{i-3}: {state}")
    
    # Define qubits to swap
    qubit1, qubit2 = -1, 0
    
    # Perform the neighbor swap
    game.swap_neighbors(qubit1, qubit2)
    
    print("\nFinal qubit states:")
    for i, state in enumerate(game.basis):
        print(f"q_{i-3}: {state}")
    
    # Verify the swap worked correctly
    if (states_approximately_equal(game.basis[qubit1+3], initial_basis[qubit2+3]) and 
        states_approximately_equal(game.basis[qubit2+3], initial_basis[qubit1+3])):
        print(f"Success: States of qubits {qubit1} and {qubit2} were swapped correctly")
    else:
        print(f"Failure: States of qubits {qubit1} and {qubit2} were not swapped correctly")
    
    # Verify system state is consistent
    game.verify_state()
    
    print("Neighbor swap test completed!")

def test_swap():
    game = SwappingGame()
    initial_basis = [state[:] for state in game.basis]  # Copy the initial qubit states
    
    print("Initial qubit states:")
    for i, state in enumerate(initial_basis):
        print(f"q_{i-3}: {state}")
    
    # Define qubits to swap
    qubit1, qubit2 = 2, -3
    
    # Perform the swap
    game.swap(qubit1, qubit2)
    
    print("\nFinal qubit states:")
    for i, state in enumerate(game.basis):
        print(f"q_{i-3}: {state}")
    
    # Verify the swap worked correctly
    if (states_approximately_equal(game.basis[qubit1+3], initial_basis[qubit2+3]) and 
        states_approximately_equal(game.basis[qubit2+3], initial_basis[qubit1+3])):
        print(f"Success: States of qubits {qubit1} and {qubit2} were swapped correctly")
    else:
        print(f"Failure: States of qubits {qubit1} and {qubit2} were not swapped correctly")
    
    # Verify system state is consistent
    game.verify_state()
    
    print("Non-neighbor swap test completed!")

test_swap_neighbors()
test_swap()

Initial qubit states:
q_-3: [0.9660581792740769, 0.2583245908883927]
q_-2: [0.9792663399701554, 0.20257698635693058]
q_-1: [0.9001130939100807, 0.43565630739336514]
q_0: [0.9252210565924197, 0.3794285129481001]
q_1: [0.8902293072181164, 0.4555126568713019]
q_2: [0.9997435844539484, 0.02264432253724967]
q_3: [0.9973726251688856, 0.07244202208473771]
Neighbor qubits -1 and 0 are successfully swapped

Final qubit states:
q_-3: [0.966058179274077, 0.25832459088839266]
q_-2: [0.9792663399701553, 0.20257698635693056]
q_-1: [0.9252210565924197, 0.3794285129481002]
q_0: [0.9001130939100807, 0.43565630739336525]
q_1: [0.8902293072181162, 0.4555126568713019]
q_2: [0.9997435844539482, 0.022644322537249666]
q_3: [0.9973726251688854, 0.07244202208473771]
Success: States of qubits -1 and 0 were swapped correctly
Verification successful: System state matches tensor product of qubit states
Neighbor swap test completed!
Initial qubit states:
q_-3: [0.9953123330619865, 0.09671276882969085]
q_-2: [0.9937

<h1> Some important notes, explanations, and visual examples of the methods inside the SwappingGame class</h1>

In [ ]:
    """
        ## Detailed Visualization of `create_initial_state()`
        
        ### Step 1: Creating 7 Random Qubit States
        
        Let's imagine we randomly generate these angles for our 7 qubits:
        - q₋₃: θ₀ = 0.2 → state = [cos(0.2), sin(0.2)] = [0.980, 0.199]
        - q₋₂: θ₁ = 0.3 → state = [cos(0.3), sin(0.3)] = [0.955, 0.296]
        - q₋₁: θ₂ = 0.1 → state = [cos(0.1), sin(0.1)] = [0.995, 0.100]
        - q₀: θ₃ = 0.4 → state = [cos(0.4), sin(0.4)] = [0.921, 0.389]
        - q₊₁: θ₄ = 0.15 → state = [cos(0.15), sin(0.15)] = [0.989, 0.150]
        - q₊₂: θ₅ = 0.25 → state = [cos(0.25), sin(0.25)] = [0.969, 0.247]
        - q₊₃: θ₆ = 0.35 → state = [cos(0.35), sin(0.35)] = [0.939, 0.343]
        
        Each qubit is represented by a 2-dimensional vector where:
        - First element: Amplitude for |0⟩ state = cos(θ)
        - Second element: Amplitude for |1⟩ state = sin(θ)
        
        Visually, the basis list would look like:
        ```
        self.basis = [
            [0.980, 0.199],  # q₋₃
            [0.955, 0.296],  # q₋₂
            [0.995, 0.100],  # q₋₁
            [0.921, 0.389],  # q₀
            [0.989, 0.150],  # q₊₁
            [0.969, 0.247],  # q₊₂
            [0.939, 0.343]   # q₊₃
        ]
        ```
        
        ### Step 2: Computing the Tensor Product
        
        Now let's see how the tensor product builds up step by step:

        1. Start with [1.0]
        
        2. Tensor with q₋₃ [0.980, 0.199]:
           ```
           [1.0] ⊗ [0.980, 0.199] = [1.0×0.980, 1.0×0.199] = [0.980, 0.199]
           ```
        
        3. Tensor with q₋₂ [0.955, 0.296]:
           ```
           [0.980, 0.199] ⊗ [0.955, 0.296] = 
           [0.980×0.955, 0.980×0.296, 0.199×0.955, 0.199×0.296] = 
           [0.936, 0.290, 0.190, 0.059]
           ```
        
        4. Tensor with q₋₁ [0.995, 0.100]:
           ```
           [0.936, 0.290, 0.190, 0.059] ⊗ [0.995, 0.100] =
           [0.936×0.995, 0.936×0.100, 0.290×0.995, 0.290×0.100, 
            0.190×0.995, 0.190×0.100, 0.059×0.995, 0.059×0.100] =
           [0.931, 0.094, 0.289, 0.029, 0.189, 0.019, 0.059, 0.006]
           ```
        
        5. Continue for all 7 qubits...
        
        The final result is a 128-element vector (2⁷ = 128), where each element corresponds to the amplitude of a specific basis state.
        
        ### Step 3: Understanding the Final System State
        
        The final system state is a 128-dimensional vector where:
        - Index 0 corresponds to |0000000⟩ (all qubits in state |0⟩)
        - Index 1 corresponds to |0000001⟩ (only q₊₃ in state |1⟩)
        - Index 2 corresponds to |0000010⟩ (only q₊₂ in state |1⟩)
        - ...
        - Index 127 corresponds to |1111111⟩ (all qubits in state |1⟩)

        Each element in this vector gives the amplitude (related to probability) of finding the system in that particular basis state.
        
        For example:
        - The amplitude for state |0000000⟩ would be:
          cos(θ₀)×cos(θ₁)×cos(θ₂)×cos(θ₃)×cos(θ₄)×cos(θ₅)×cos(θ₆)
          
        - The amplitude for state |0000001⟩ would be:
          cos(θ₀)×cos(θ₁)×cos(θ₂)×cos(θ₃)×cos(θ₄)×cos(θ₅)×sin(θ₆)
        
        - The amplitude for state |1111111⟩ would be:
          sin(θ₀)×sin(θ₁)×sin(θ₂)×sin(θ₃)×sin(θ₄)×sin(θ₅)×sin(θ₆)
        
        This 128-dimensional vector `self.system_state` completely describes the quantum state of our 7-qubit system. The tensor product ensures that all possible combinations of the individual qubit states are represented with the correct amplitudes.
    """


In [ ]:
"""


## The `find_path` Method Explained

The `find_path` method is used to find a path between two qubits in your quantum system, which is necessary for the `swap` operation to work when swapping non-neighboring qubits.

### Purpose
When you want to swap two qubits that aren't directly connected, you need to find a sequence of neighboring swaps that will effectively move one qubit's state to another's position.


### Visual Example

Let's visualize this with your qubit layout:

```
q_{-3} -- q_{-1} -- q_{-2}
            |
          q_{0}
            |
          q_{+1} -- q_{+2} -- q_{+3}
```

Imagine we want to find a path from `q_{-3}` to `q_{+3}`.

#### Step 1: Converting external indices to internal indices
```
start = -3 → internal_start = -3 + 3 = 0
end = 3 → internal_end = 3 + 3 = 6
```

#### Step 2: Initialize BFS
- Initialize `queue = [0]` (starting with qubit `q_{-3}`)
- Initialize `visited = {0}` (marking `q_{-3}` as visited)
- Initialize `parents = {}` (empty dictionary to track the path)

#### Step 3: BFS Execution

Let's trace through the execution:

1. **Iteration 1:**
   - Remove `0` from queue → `current = 0`
   - Check if `current == internal_end` (if `0 == 6`) → False
   - Get neighbors of `0` → `[2]` (qubit `q_{-1}`)
   - For neighbor `2`:
     - Not visited, so add to queue: `queue = [2]`
     - Mark as visited: `visited = {0, 2}`
     - Set parent: `parents = {2: 0}`

   Visual state:
   ```
   [q_{-3}] -- q_{-1} -- q_{-2}
   (visited)    |
              q_{0}
                |
              q_{+1} -- q_{+2} -- q_{+3}
   ```

2. **Iteration 2:**
   - Remove `2` from queue → `current = 2`
   - Check if `current == internal_end` → False
   - Get neighbors of `2` → `[0, 1, 3]` (qubits `q_{-3}`, `q_{-2}`, and `q_{0}`)
   - For neighbor `0`: already visited, skip
   - For neighbor `1`:
     - Not visited, so add to queue: `queue = [3, 1]`
     - Mark as visited: `visited = {0, 2, 1}`
     - Set parent: `parents = {2: 0, 1: 2}`
   - For neighbor `3`:
     - Not visited, so add to queue: `queue = [1, 3]`
     - Mark as visited: `visited = {0, 2, 1, 3}`
     - Set parent: `parents = {2: 0, 1: 2, 3: 2}`

   Visual state:
   ```
   [q_{-3}] -- [q_{-1}] -- [q_{-2}]
   (visited)  (visited)   (visited)
                  |
               [q_{0}]
              (visited)
                  |
                q_{+1} -- q_{+2} -- q_{+3}
   ```

3. **Iteration 3:**
   - Remove `1` from queue → `current = 1`
   - Check if `current == internal_end` → False
   - Get neighbors of `1` → `[2]` (already visited, skip)

4. **Iteration 4:**
   - Remove `3` from queue → `current = 3`
   - Check if `current == internal_end` → False
   - Get neighbors of `3` → `[2, 4]` (qubit `q_{-1}` already visited, qubit `q_{+1}` not visited)
   - For neighbor `4`:
     - Not visited, so add to queue: `queue = [4]`
     - Mark as visited: `visited = {0, 2, 1, 3, 4}`
     - Set parent: `parents = {2: 0, 1: 2, 3: 2, 4: 3}`

   Visual state:
   ```
   [q_{-3}] -- [q_{-1}] -- [q_{-2}]
   (visited)  (visited)   (visited)
                  |
               [q_{0}]
              (visited)
                  |
              [q_{+1}] -- q_{+2} -- q_{+3}
              (visited)
   ```

5. **Iteration 5:**
   - Remove `4` from queue → `current = 4`
   - Check if `current == internal_end` → False
   - Get neighbors of `4` → `[3, 5]` (qubit `q_{0}` already visited, qubit `q_{+2}` not visited)
   - For neighbor `5`:
     - Not visited, so add to queue: `queue = [5]`
     - Mark as visited: `visited = {0, 2, 1, 3, 4, 5}`
     - Set parent: `parents = {2: 0, 1: 2, 3: 2, 4: 3, 5: 4}`

   Visual state:
   ```
   [q_{-3}] -- [q_{-1}] -- [q_{-2}]
   (visited)  (visited)   (visited)
                  |
               [q_{0}]
              (visited)
                  |
              [q_{+1}] -- [q_{+2}] -- q_{+3}
              (visited)  (visited)
   ```

6. **Iteration 6:**
   - Remove `5` from queue → `current = 5`
   - Check if `current == internal_end` → False
   - Get neighbors of `5` → `[4, 6]` (qubit `q_{+1}` already visited, qubit `q_{+3}` not visited)
   - For neighbor `6`:
     - Not visited, so add to queue: `queue = [6]`
     - Mark as visited: `visited = {0, 2, 1, 3, 4, 5, 6}`
     - Set parent: `parents = {2: 0, 1: 2, 3: 2, 4: 3, 5: 4, 6: 5}`

   Visual state:
   ```
   [q_{-3}] -- [q_{-1}] -- [q_{-2}]
   (visited)  (visited)   (visited)
                  |
               [q_{0}]
              (visited)
                  |
              [q_{+1}] -- [q_{+2}] -- [q_{+3}]
              (visited)  (visited)  (visited)
   ```

7. **Iteration 7:**
   - Remove `6` from queue → `current = 6`
   - Check if `current == internal_end` → True (We found the target!)
   - Reconstruct path:
     - Initialize `path = [6]`
     - While `current != internal_start`:
       - `current = parents[current]` = `parents[6]` = `5`
       - Insert at beginning: `path = [5, 6]`
       - `current = parents[current]` = `parents[5]` = `4`
       - Insert at beginning: `path = [4, 5, 6]`
       - `current = parents[current]` = `parents[4]` = `3`
       - Insert at beginning: `path = [3, 4, 5, 6]`
       - `current = parents[current]` = `parents[3]` = `2`
       - Insert at beginning: `path = [2, 3, 4, 5, 6]`
       - `current = parents[current]` = `parents[2]` = `0`
       - Insert at beginning: `path = [0, 2, 3, 4, 5, 6]`
       - Now `current == internal_start`, so we stop
     - Convert back to external indices: `[index - 3 for index in path]`
       = `[-3, -1, 0, 1, 2, 3]`
     - Return the path

#### Step 4: Result
The function returns the path `[-3, -1, 0, 1, 2, 3]`, which represents the sequence:
```
q_{-3} → q_{-1} → q_{0} → q_{+1} → q_{+2} → q_{+3}
```

This is exactly the shortest path you'd need to follow to move from `q_{-3}` to `q_{+3}` via only connected qubits!

### Practical Application in Your Code

When `swap(2, -3)` is called (swapping `q_{+2}` and `q_{-3}`), the `find_path` method would find the path:
```
q_{+2} → q_{+1} → q_{0} → q_{-1} → q_{-3}
```

Then the `swap` method uses this path to perform a sequence of neighbor swaps:
1. Swap `q_{+2}` and `q_{+1}`
2. Swap `q_{+1}` and `q_{0}`
3. Swap `q_{0}` and `q_{-1}`
4. Swap `q_{-1}` and `q_{-3}`

These swaps effectively move the state of `q_{+2}` to `q_{-3}` and vice versa.

### Key Concepts in BFS

1. **Queue**: Ensures we explore nodes in order of their distance from the start
2. **Visited Set**: Prevents revisiting nodes and creating infinite loops
3. **Parents Dictionary**: Tracks how we reached each node, allowing us to reconstruct the path

This breadth-first search is guaranteed to find the shortest path between any two qubits, which is critical for minimizing the number of swap operations needed.
"""

In [ ]:
"""
## The `swap` Method Explained

The `swap` method allows you to swap the quantum states of any two qubits in your system, even if they aren't directly connected. It uses a sequence of neighbor swaps to accomplish this.

### Purpose
This method enables swapping the quantum states of any two qubits, regardless of their position in the connectivity graph.

### How it Works
Let's break down the method step by step:

```python
def swap(self, i, j):
    if not (-3 <= i <= 3 and -3 <= j <= 3):
        raise ValueError("Qubit indices must be between -3 and 3")
    if i == j:
        return  # Nothing to do if swapping the same qubit
    # Convert from external indices (-3 to 3) to internal indices (0 to 6)
    internal_i = i + 3
    internal_j = j + 3
    
    # Save the initial states before swapping
    initial_state_i = self.basis[internal_i][:]
    initial_state_j = self.basis[internal_j][:]
    path = self.find_path(i, j)
    if path is None:
        raise ValueError(f"No path exists between qubits {i} and {j}")
    # Perform swaps along the path
    for k in range(len(path) - 1):
        self.swap_neighbors(path[k], path[k+1])
    
    # Fix the swap by directly setting the values at the endpoints
    self.basis[internal_i] = initial_state_j
    self.basis[internal_j] = initial_state_i
    
    # Recalculate the system state to ensure consistency
    self.system_state = self.tensor_product(self.basis)
    print(f"States of qubits {i} and {j} swapped.")
```

### Visual Example

Let's use the same qubit layout:

```
q_{-3} -- q_{-1} -- q_{-2}
            |
          q_{0}
            |
          q_{+1} -- q_{+2} -- q_{+3}
```

Imagine we want to swap qubits `q_{-3}` and `q_{+2}` (i.e., `swap(-3, 2)`).

#### Step 1: Validation and Index Conversion
```python
if not (-3 <= i <= 3 and -3 <= j <= 3):
    raise ValueError("Qubit indices must be between -3 and 3")
if i == j:
    return  # Nothing to do if swapping the same qubit
# Convert from external indices (-3 to 3) to internal indices (0 to 6)
internal_i = i + 3
internal_j = j + 3
```

First, we ensure the qubit indices are valid and not the same. Then, we convert from the external indices that users provide (-3 to 3) to internal indices (0 to 6) that the code uses:
- `i = -3` → `internal_i = -3 + 3 = 0`
- `j = 2` → `internal_j = 2 + 3 = 5`

#### Step 2: Save Initial Qubit States
```python
# Save the initial states before swapping
initial_state_i = self.basis[internal_i][:]
initial_state_j = self.basis[internal_j][:]
```

Before we start swapping, we save the quantum states of both qubits. Let's say:
- `q_{-3}` state is `[0.98, 0.20]` (mostly |0⟩)
- `q_{+2}` state is `[0.50, 0.87]` (more |1⟩ than |0⟩)

We save:
- `initial_state_i = [0.98, 0.20]`
- `initial_state_j = [0.50, 0.87]`

#### Step 3: Find a Path Between Qubits
```python
path = self.find_path(i, j)
if path is None:
    raise ValueError(f"No path exists between qubits {i} and {j}")
```

We use the `find_path` method to determine a sequence of neighboring qubits that connect our start and end qubits. For our example, the path returned would be:
```
path = [-3, -1, 0, 1, 2]
```

This means we'll need to traverse: `q_{-3} → q_{-1} → q_{0} → q_{+1} → q_{+2}`

#### Step 4: Perform Neighbor Swaps Along the Path
```python
# Perform swaps along the path
for k in range(len(path) - 1):
    self.swap_neighbors(path[k], path[k+1])
```

Now, we perform a series of neighbor swaps to move the states along the path:

1. First swap: `swap_neighbors(-3, -1)`
   - Before swap:
     ```
     q_{-3}=[0.98,0.20] -- q_{-1}=[0.90,0.44] -- q_{-2}=[0.77,0.64]
                                |
                            q_{0}=[0.86,0.51]
                                |
                            q_{+1}=[0.92,0.39] -- q_{+2}=[0.50,0.87] -- q_{+3}=[0.95,0.30]
     ```
   - After swap:
     ```
     q_{-3}=[0.90,0.44] -- q_{-1}=[0.98,0.20] -- q_{-2}=[0.77,0.64]
                                |
                            q_{0}=[0.86,0.51]
                                |
                            q_{+1}=[0.92,0.39] -- q_{+2}=[0.50,0.87] -- q_{+3}=[0.95,0.30]
     ```

2. Second swap: `swap_neighbors(-1, 0)`
   - After swap:
     ```
     q_{-3}=[0.90,0.44] -- q_{-1}=[0.86,0.51] -- q_{-2}=[0.77,0.64]
                                |
                            q_{0}=[0.98,0.20]
                                |
                            q_{+1}=[0.92,0.39] -- q_{+2}=[0.50,0.87] -- q_{+3}=[0.95,0.30]
     ```

3. Third swap: `swap_neighbors(0, 1)`
   - After swap:
     ```
     q_{-3}=[0.90,0.44] -- q_{-1}=[0.86,0.51] -- q_{-2}=[0.77,0.64]
                                |
                            q_{0}=[0.92,0.39]
                                |
                            q_{+1}=[0.98,0.20] -- q_{+2}=[0.50,0.87] -- q_{+3}=[0.95,0.30]
     ```

4. Fourth swap: `swap_neighbors(1, 2)`
   - After swap:
     ```
     q_{-3}=[0.90,0.44] -- q_{-1}=[0.86,0.51] -- q_{-2}=[0.77,0.64]
                                |
                            q_{0}=[0.92,0.39]
                                |
                            q_{+1}=[0.50,0.87] -- q_{+2}=[0.98,0.20] -- q_{+3}=[0.95,0.30]
     ```

At this point, we've successfully moved the state of `q_{-3}` to `q_{+2}`, but we've also moved other states around in the process. Notice that `q_{-3}` doesn't have the state we want yet - it has `[0.90,0.44]` instead of `[0.50,0.87]`.

#### Step 5: Fix the Swap by Directly Setting Endpoint Values
```python
# Fix the swap by directly setting the values at the endpoints
self.basis[internal_i] = initial_state_j
self.basis[internal_j] = initial_state_i
```

This is a crucial step! After the neighbor swaps, the states have moved along the path, but we need to ensure the end result is exactly what we want. We directly set:
- `self.basis[0]` (q_{-3}) = `initial_state_j` = `[0.50, 0.87]`
- `self.basis[5]` (q_{+2}) = `initial_state_i` = `[0.98, 0.20]`

Final state after fixing:
```
q_{-3}=[0.50,0.87] -- q_{-1}=[0.86,0.51] -- q_{-2}=[0.77,0.64]
                            |
                        q_{0}=[0.92,0.39]
                            |
                        q_{+1}=[0.50,0.87] -- q_{+2}=[0.98,0.20] -- q_{+3}=[0.95,0.30]
```

#### Step 6: Recalculate the System State
```python
# Recalculate the system state to ensure consistency
self.system_state = self.tensor_product(self.basis)
```

Finally, we recalculate the full 128-dimensional quantum state of the 7-qubit system by computing the tensor product of all the individual qubit states. This ensures our system state is consistent with the individual qubit states.

#### Step 7: Confirmation
```python
print(f"States of qubits {i} and {j} swapped.")
```

The method confirms the operation with a message: "States of qubits -3 and 2 swapped."

### Why the Fix Step is Necessary

You might wonder why we need step 5 (directly setting the endpoint values) when we've already swapped along the path. There are two main reasons:

1. **Precision Issues**: Quantum operations can accumulate numerical errors. Direct assignment ensures the exact values.

2. **Fixing Intermediate States**: The neighbor swaps might affect intermediate qubits in unintended ways. For instance, in our example, after the swaps, `q_{+1}` ended up with the same state as `q_{-3}` originally had. This is a side effect we don't want.

By directly assigning the states at the endpoints, we ensure the result is exactly what we intended: swapping only the states of qubits `i` and `j`.

### Key Concepts in the Swap Method

1. **Path Finding**: Using `find_path` to determine how to move between non-adjacent qubits.
2. **Sequential Neighbor Swaps**: Performing a series of adjacent swaps to move states along the path.
3. **Direct State Assignment**: Ensuring the final state is exactly as desired by directly setting the endpoint states.
4. **System State Recalculation**: Maintaining consistency between individual qubit states and the full system state.

This method enables swapping any two qubits while respecting the physical connectivity constraints of your quantum system, which is crucial for implementing quantum algorithms on hardware with limited connectivity.
"""

In [ ]:
"""

## A Simplified Explanation of `update_qubit_states()`

Imagine we have just 2 qubits instead of 7, to make this easier to visualize.

After operations like CNOT, the system state might look like this:
```
system_state = [0.7, 0.0, 0.0, 0.7]
```

This represents:
- 0.7 probability amplitude for |00⟩
- 0.0 probability amplitude for |01⟩
- 0.0 probability amplitude for |10⟩
- 0.7 probability amplitude for |11⟩

Now, what is the state of qubit 0 by itself?

The `update_qubit_states()` method figures this out by:

1. Finding all states where qubit 0 is |0⟩:
   - |00⟩ has amplitude 0.7
   - |01⟩ has amplitude 0.0
   - Total probability: 0.7² + 0.0² = 0.49

2. Finding all states where qubit 0 is |1⟩:
   - |10⟩ has amplitude 0.0
   - |11⟩ has amplitude 0.7
   - Total probability: 0.0² + 0.7² = 0.49

3. The state of qubit 0 is therefore approximately:
   - Amplitude for |0⟩ = √0.49 = 0.7
   - Amplitude for |1⟩ = √0.49 = 0.7
   - So qubit 0's state is [0.7, 0.7]

The code does this for each qubit in your system:

```python
# For qubit 0:
prob_0 = 0.7² + 0.0² = 0.49  # From states |00⟩ and |01⟩
prob_1 = 0.0² + 0.7² = 0.49  # From states |10⟩ and |11⟩
amplitude_0 = √0.49 = 0.7
amplitude_1 = √0.49 = 0.7
self.basis[0] = [0.7, 0.7]

# For qubit 1:
prob_0 = 0.7² + 0.0² = 0.49  # From states |00⟩ and |10⟩
prob_1 = 0.0² + 0.7² = 0.49  # From states |01⟩ and |11⟩
amplitude_0 = √0.49 = 0.7
amplitude_1 = √0.49 = 0.7
self.basis[1] = [0.7, 0.7]
```

Going back to the 7-qubit system:

1. The system state has 128 amplitudes (for all possible 7-qubit states)
2. For each qubit (0-6), the method:
   - Adds up probabilities for all states where that qubit is |0⟩
   - Adds up probabilities for all states where that qubit is |1⟩
   - Takes square roots to get amplitudes
   - Updates that qubit's state in the `basis` list

Think of it as answering: "If I only look at qubit X by itself, what state would it appear to be in?"
"""